In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data_re_id.npy');
path_to_labels = path.join(root_folder,'data/train_labels_re_id.npy');
#path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [17]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);

torch_labels = torch.tensor(labels).int()

torch_data = torch.zeros((torch_labels.nelement(), 1 , 72)).int()

for i in range(torch_data.size()[0]):
    torch_data[i, :] = torch_data[i, :] + i;
nb_of_classes = torch_labels.unique().size()[0]
if (len(torch_labels.size()) == 2):
    torch_labels = torch.squeeze(torch_labels)
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [ ]:
for i in range(torch_labels.nelement()):
    for j in range(test_data.data.size()[0]):
        res  = (torch_data[i, :] == test_data.data[j, :].int()).numpy()
        if(np.sum(res) > 0):
            print(torch_labels[i] == test_data.labels[j].int(),torch_labels[i], test_data.labels[j])


tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(15, dtype=torch.int32) tensor(15.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(16, dtype=torch.int32) tensor(16.)
tensor(1, dtype=torch.uint8) tensor(13, dtype=torch.int32) tensor(13.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(16, dtype=torch.int32) tensor(16.)
tensor(1, dtype=torch.uint8) tensor(11, dtype=torch.int32) tensor(11.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(16, dtype=torch.int32) tensor(16.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.

tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(13, dtype=torch.int32) tensor(13.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(15, dtype=torch.int32) tensor(15.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(11, dtype=torch.int32) tensor(11.)
tensor(1, dtype=torc

tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(15, dtype=torch.int32) tensor(15.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(12, dtype=torch.int32) tensor(12.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(11, dtype=torch.int32) tensor(11.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.ui

tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(15, dtype=torch.int32) tensor(15.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(15, dtype=torch.int32) tensor(15.)
tensor(1, dtype=torch.uint8) tensor(11, dtype=torch.int32) tensor(11.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(15, dtype=torch.int32) tensor(15.)
tensor(1, dtype=torch.

tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(13, dtype=torch.int32) tensor(13.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(11, dtype=torch.int32) tensor(11.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(12, dtype=torch.int32) tensor(12.)
tensor(1, dtype=torch.uint8) tensor(13, dtype=torch.int32) tensor(13.)
tensor(1, dtype=torch.uint8) tensor(13, dtype=torch.int32) tensor(13.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(12, dtype=torch.int32) tensor(12.)
tensor(1, dtype=torch.uint8) tensor(13, dtype=torch.int32) tensor(13.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, dtype=torch.uint8) tensor(14, dtype=torch.int32) tensor(14.)
tensor(1, 

...

tensor(1, dtype=torch.uint8) tensor(68, dtype=torch.int32) tensor(68.)
tensor(1, dtype=torch.uint8) tensor(74, dtype=torch.int32) tensor(74.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(72, dtype=torch.int32) tensor(72.)
tensor(1, dtype=torch.uint8) tensor(75, dtype=torch.int32) tensor(75.)
tensor(1, dtype=torch.uint8) tensor(67, dtype=torch.int32) tensor(67.)
tensor(1, dtype=torch.uint8) tensor(70, dtype=torch.int32) tensor(70.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(73, dtype=torch.int32) tensor(73.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(68, dtype=torch.int32) tensor(68.)
tensor(1, dtype=torch.uint8) tensor(77, dtype=torch.int32) tensor(77.)
tensor(1, dtype=torch.uint8) tensor(70, dtype=torch.int32) tensor(70.)
tensor(1, dtype=torch.uint8) tensor(68, dtype=torch.int32) tensor(68.)
tensor

tensor(1, dtype=torch.uint8) tensor(70, dtype=torch.int32) tensor(70.)
tensor(1, dtype=torch.uint8) tensor(77, dtype=torch.int32) tensor(77.)
tensor(1, dtype=torch.uint8) tensor(74, dtype=torch.int32) tensor(74.)
tensor(1, dtype=torch.uint8) tensor(73, dtype=torch.int32) tensor(73.)
tensor(1, dtype=torch.uint8) tensor(67, dtype=torch.int32) tensor(67.)
tensor(1, dtype=torch.uint8) tensor(75, dtype=torch.int32) tensor(75.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(68, dtype=torch.int32) tensor(68.)
tensor(1, dtype=torch.uint8) tensor(75, dtype=torch.int32) tensor(75.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(73, dtype=torch.int32) tensor(73.)
tensor(1, dtype=torch.uint8) tensor(71, dtype=torch.int32) tensor(71.)
tensor(1, dtype=torch.uint8) tensor(75, dtype=torch.int32) tensor(75.)
tensor

tensor(1, dtype=torch.uint8) tensor(77, dtype=torch.int32) tensor(77.)
tensor(1, dtype=torch.uint8) tensor(77, dtype=torch.int32) tensor(77.)
tensor(1, dtype=torch.uint8) tensor(70, dtype=torch.int32) tensor(70.)
tensor(1, dtype=torch.uint8) tensor(69, dtype=torch.int32) tensor(69.)
tensor(1, dtype=torch.uint8) tensor(70, dtype=torch.int32) tensor(70.)
tensor(1, dtype=torch.uint8) tensor(70, dtype=torch.int32) tensor(70.)
tensor(1, dtype=torch.uint8) tensor(69, dtype=torch.int32) tensor(69.)
tensor(1, dtype=torch.uint8) tensor(69, dtype=torch.int32) tensor(69.)
tensor(1, dtype=torch.uint8) tensor(76, dtype=torch.int32) tensor(76.)
tensor(1, dtype=torch.uint8) tensor(69, dtype=torch.int32) tensor(69.)
tensor(1, dtype=torch.uint8) tensor(69, dtype=torch.int32) tensor(69.)
tensor(1, dtype=torch.uint8) tensor(69, dtype=torch.int32) tensor(69.)
tensor(1, dtype=torch.uint8) tensor(76, dtype=torch.int32) tensor(76.)
tensor(1, dtype=torch.uint8) tensor(76, dtype=torch.int32) tensor(76.)
tensor

tensor(1, dtype=torch.uint8) tensor(86, dtype=torch.int32) tensor(86.)
tensor(1, dtype=torch.uint8) tensor(79, dtype=torch.int32) tensor(79.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(84, dtype=torch.int32) tensor(84.)
tensor(1, dtype=torch.uint8) tensor(86, dtype=torch.int32) tensor(86.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(78, dtype=torch.int32) tensor(78.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(79, dtype=torch.int32) tensor(79.)
tensor(1, dtype=torch.uint8) tensor(89, dtype=torch.int32) tensor(89.)
tensor(1, dtype=torch.uint8) tensor(87, dtype=torch.int32) tensor(87.)
tensor(1, dtype=torch.uint8) tensor(78, dtype=torch.int32) tensor(78.)
tensor

tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(78, dtype=torch.int32) tensor(78.)
tensor(1, dtype=torch.uint8) tensor(90, dtype=torch.int32) tensor(90.)
tensor(1, dtype=torch.uint8) tensor(89, dtype=torch.int32) tensor(89.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(86, dtype=torch.int32) tensor(86.)
tensor(1, dtype=torch.uint8) tensor(80, dtype=torch.int32) tensor(80.)
tensor(1, dtype=torch.uint8) tensor(90, dtype=torch.int32) tensor(90.)
tensor(1, dtype=torch.uint8) tensor(82, dtype=torch.int32) tensor(82.)
tensor(1, dtype=torch.uint8) tensor(87, dtype=torch.int32) tensor(87.)
tensor(1, dtype=torch.uint8) tensor(84, dtype=torch.int32) tensor(84.)
tensor(1, dtype=torch.uint8) tensor(92, dtype=torch.int32) tensor(92.)
tensor(1, dtype=torch.uint8) tensor(90, dtype=torch.int32) tensor(90.)
tensor

In [25]:
        res  = (torch_data[i, :] == test_data.data[j, :]).numpy()


RuntimeError: Expected object of scalar type Int but got scalar type Float for argument #2 'other'